In [ ]:
#1-person/human, 0-spoof

In [2]:
import time
import glob
from IPython.display import Audio
import logging
from concurrent.futures import ThreadPoolExecutor
from joblib import Parallel, delayed
import time
import logging
from IPython.display import Audio
from tqdm import tqdm
import subprocess
import pandas as pd

from IDRnD.utils import *
from IDRnD.augmentations import *
from IDRnD.dataset import *
from IDRnD.resnet import resnet50
from IDRnD.focalloss import FocalLoss
import numpy as np

import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, CyclicLR, ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

from sklearn.model_selection import StratifiedKFold, train_test_split

%reload_ext autoreload
%autoreload 2
%matplotlib inline

seed_everything(0)
logging.basicConfig(level=logging.DEBUG, filename="logs/logs.log",
                    filemode="w+")

In [3]:
X, y = get_train_data()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [4]:
train_transform = transforms.Compose([
    #RandomParameter(VTLP, [[0.8, 1.2]], p=0.5),
    #MinMaxChunkScaler(),
    #Normalize(),
    #RandomParameter(RandomNoise, [[0.01, 0.1]]),
    #RandomParameter(Shift, [[2000, 32000]]),
    #RandomParameter(TimeStretch, [[0.75, 1.3]]),
    #RandomParameter(PitchShift, [[-8, 8]]),
    #RandomParameter(Distortion, [[-1, -0.3], [.3, 1.]]),
    ToMellSpec(n_mels=128),
    #GetMFCC(),
    #PadOrClip(320),
    #Normalize(),
    #ToTensor(),
    #transforms.ToTensor(),
])

In [5]:
old_spoof1 = pd.read_csv("/data_spoof/CM_protocol/cm_train.trn", sep = ' ', names=["folder", "filename", "wat", "class"])
old_spoof2 = pd.read_csv("/data_spoof/CM_protocol/cm_evaluation.ndx", sep = ' ', names=["folder", "filename", "wat", "class"])
old_spoof3 = pd.read_csv("/data_spoof/CM_protocol/cm_develop.ndx", sep = ' ', names=["folder", "filename", "wat", "class"])
old_spoof = pd.concat([old_spoof1, old_spoof2, old_spoof3])
old_spoof.reset_index(drop=True, inplace=True)

In [6]:
base_dir = "/data_spoof/wav/"
pathes = np.array(old_spoof[["folder", "filename"]].apply(lambda x:os.path.join(base_dir, *x)+".wav", axis=1))
maping = {"spoof":0, "human":1}
classes = np.array(old_spoof["class"].apply(lambda x: maping[x]))

old_spoof_dataset = BaseDatasetScipy(pathes, classes, train_transform)

In [35]:
class AudioProcessingUnit:
    def __init__(self):
        self.video_folder = "/data_common_voices/en/clips"
        self.save_folder = "/data_common_voices/en/clips_wavs"

    def convert_to_wav(self, video_name: str):
        """
        Converting webm to wav file .
        """
        video_path = f"{self.video_folder}/{video_name}"

        out = subprocess.Popen(['ffmpeg', "-threads", "16", '-i', video_path, "-vn", "-acodec", "pcm_s16le",
                                "-ac", "1", "-ar", "16000", "-f", "wav",  f'{self.save_folder}/{video_name}.wav'],
                               stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        #out.wait()

In [36]:
apu = AudioProcessingUnit()

In [4]:
train_dataset = SimpleMelDataset(X, y, "../data/files/raw_mels/", train_transform)
#val_dataset = Base_Dataset(X_val, y_val, train_transform)

In [ ]:
base_path = "/data_common_voices/en/clips/"
common_voice_files = os.listdir(base_path)
common = [os.path.join(base_path, file) for file in common_voice_files]
common_dataset = BaseDatasetScipy(common, np.ones_like(common, dtype=np.int16), train_transform)

In [27]:
Parallel(n_jobs=16, verbose=5, backend="multiprocessing")(delayed(apu.convert_to_wav)(file) for file in common_voice_files)

[Parallel(n_jobs=200)]: Using backend ThreadingBackend with 200 concurrent workers.
[Parallel(n_jobs=200)]: Done  50 tasks      | elapsed:    5.8s
[Parallel(n_jobs=200)]: Done 248 tasks      | elapsed:    7.8s
[Parallel(n_jobs=200)]: Done 482 tasks      | elapsed:   11.7s
[Parallel(n_jobs=200)]: Done 752 tasks      | elapsed:   14.9s
[Parallel(n_jobs=200)]: Done 1058 tasks      | elapsed:   19.0s
[Parallel(n_jobs=200)]: Done 1400 tasks      | elapsed:   23.6s
[Parallel(n_jobs=200)]: Done 1778 tasks      | elapsed:   28.4s
[Parallel(n_jobs=200)]: Done 2192 tasks      | elapsed:   34.0s
[Parallel(n_jobs=200)]: Done 2642 tasks      | elapsed:   34.8s


KeyboardInterrupt: 

In [6]:
def move_file(file):
    new_name = os.path.join(base_path, file.replace('.mp3.mp3', ".wav"))
    old_name = os.path.join(base_path, file)
    os.rename(old_name, new_name)

In [ ]:
print(len(common_dataset))
Parallel(n_jobs=16, verbose=5)(delayed(move_file)(file) for file in common_voice_files)

945939


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  44 tasks      | elapsed:    0.5s
[Parallel(n_jobs=16)]: Done 3376 tasks      | elapsed:    1.0s
[Parallel(n_jobs=16)]: Done 27134 tasks      | elapsed:    2.2s
[Parallel(n_jobs=16)]: Done 86750 tasks      | elapsed:    3.8s
[Parallel(n_jobs=16)]: Done 139922 tasks      | elapsed:  1.6min
[Parallel(n_jobs=16)]: Done 183663 tasks      | elapsed:  4.1min
[Parallel(n_jobs=16)]: Done 233574 tasks      | elapsed:  6.7min
[Parallel(n_jobs=16)]: Done 280165 tasks      | elapsed:  9.2min
[Parallel(n_jobs=16)]: Done 337940 tasks      | elapsed: 12.8min
[Parallel(n_jobs=16)]: Done 412533 tasks      | elapsed: 17.1min
[Parallel(n_jobs=16)]: Done 499537 tasks      | elapsed: 22.5min
[Parallel(n_jobs=16)]: Done 569473 tasks      | elapsed: 26.8min
[Parallel(n_jobs=16)]: Done 637694 tasks      | elapsed: 30.9min
[Parallel(n_jobs=16)]: Done 735607 tasks      | elapsed: 36.9min
[Parallel(n_jobs=16)

In [10]:
def precalculate_and_save_one_epoch(folder, run, dataset):
    for sample in tqdm(range(len(dataset))):
        k = dataset[sample][0]
        name = f"{run}_{sample}"
        path = os.path.join(folder, name)
        np.save(path, k)

In [11]:
def save_sample(sample, dataset, run, folder):
    try:
        k = dataset[sample][0]
        name = dataset.X[sample].split("/")[-1].split(".")[0]
        #name = f"{run}_{sample}"
        path = os.path.join(folder, name)
        np.save(path, k)
    except Exception as exp:
        print(exp)

In [12]:
def precalculate_and_save_one_epoch_parallel(run, folder, dataset):
    before = time.time()
    with ThreadPoolExecutor(max_workers=16) as executor:
        for sample in range(len(dataset)):
            future = executor.submit(save_sample, sample, dataset, run, folder)
    
    print(time.time() - before)

In [13]:
def precalculate_and_save_one_epoch_joblib(run, folder, dataset):
    Parallel(n_jobs=16, verbose=5, backend="multiprocessing")(delayed(save_sample)(sample, dataset, run, folder)
                        for sample in range(len(dataset)))

!rm -rf ../data/files/raw_mels
!rm -rf ../data/files/raw_mels_val

In [14]:
librosa.core.load("/data_common_voices/en/clips/common_voice_en_495975.mp3")

(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 22050)

In [14]:
for num in range(0, 1):
    precalculate_and_save_one_epoch_joblib(num, "../data/files/raw_mels", old_spoof_dataset)
    #precalculate_and_save_one_epoch_joblib(num, "../data/files/raw_mels_val", val_dataset)

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  48 tasks      | elapsed:    2.8s
[Parallel(n_jobs=16)]: Done 162 tasks      | elapsed:    8.5s
[Parallel(n_jobs=16)]: Done 288 tasks      | elapsed:   17.1s
[Parallel(n_jobs=16)]: Done 450 tasks      | elapsed:   27.9s
[Parallel(n_jobs=16)]: Done 648 tasks      | elapsed:   40.7s
[Parallel(n_jobs=16)]: Done 882 tasks      | elapsed:   56.4s
[Parallel(n_jobs=16)]: Done 1152 tasks      | elapsed:  1.2min
[Parallel(n_jobs=16)]: Done 1458 tasks      | elapsed:  1.6min
[Parallel(n_jobs=16)]: Done 1800 tasks      | elapsed:  2.0min
[Parallel(n_jobs=16)]: Done 2178 tasks      | elapsed:  2.4min
[Parallel(n_jobs=16)]: Done 2592 tasks      | elapsed:  2.9min
[Parallel(n_jobs=16)]: Done 3042 tasks      | elapsed:  3.4min
[Parallel(n_jobs=16)]: Done 3528 tasks      | elapsed:  3.9min
[Parallel(n_jobs=16)]: Done 4050 tasks      | elapsed:  4.5min
[Parallel(n_jobs=16)]: Done 4608 tas

In [41]:
len(common_dataset)

71781

In [ ]:
len(os.listdir("../data/files/raw_mels/"))